In [9]:
#Single hidden layer neural network
#do the essential imports
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

#essential functions
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

#create the model function
def model(X, w_h, w_o):
    h = tf.nn.sigmoid(tf.matmul(X, w_h))
    #don't take softmax here cost function does it for us
    o = tf.matmul(h, w_o)
    return o
    
#read the dataset
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
#create the placeholders and ops
#X's shape is batch X feature size
#Y's shape is batch X output size
FEATURE_SIZE      = 784
HIDDEN_LAYER_SIZE = 40
X                 = tf.placeholder("float", [None, FEATURE_SIZE])
Y                 = tf.placeholder("float", [None, 10])
w_h               = init_weights([FEATURE_SIZE, HIDDEN_LAYER_SIZE])
w_o               = init_weights([HIDDEN_LAYER_SIZE, 10])

# X = tf.placeholder("float", [None, 784])
# Y = tf.placeholder("float", [None, 10])

# w_h = init_weights([784, 625]) # create symbolic variables
# w_o = init_weights([625, 10])

# py_x = model(X, w_h, w_o)

# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y)) # compute costs
# train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost) # construct an optimizer
# predict_op = tf.argmax(py_x, 1)
pr_y              = model(X, w_h, w_o)
cost              = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pr_y, labels=Y))
train_op          = tf.train.GradientDescentOptimizer(0.05).minimize(cost)
predict_op        = tf.argmax(pr_y, 1)



with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(100):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            sess.run(train_op, feed_dict={X:trX[start:end], Y:trY[start:end]})
        if i%1==0:
            predictions = sess.run(predict_op, feed_dict={X:teX})
            actuals     = np.argmax(teY, axis=1)
            accuracy    = np.mean(predictions==actuals)
            print("epoch " + str(i) + " Accuracy:- " + str(accuracy))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
epoch 0 Accuracy:- 0.3908
epoch 1 Accuracy:- 0.6534
epoch 2 Accuracy:- 0.7926
epoch 3 Accuracy:- 0.8434
epoch 4 Accuracy:- 0.8686
epoch 5 Accuracy:- 0.8851
epoch 6 Accuracy:- 0.8931
epoch 7 Accuracy:- 0.8987
epoch 8 Accuracy:- 0.9033
epoch 9 Accuracy:- 0.907
epoch 10 Accuracy:- 0.9103
epoch 11 Accuracy:- 0.9129
epoch 12 Accuracy:- 0.9146
epoch 13 Accuracy:- 0.9167
epoch 14 Accuracy:- 0.9187
epoch 15 Accuracy:- 0.9201
epoch 16 Accuracy:- 0.9211
epoch 17 Accuracy:- 0.923
epoch 18 Accuracy:- 0.9238
epoch 19 Accuracy:- 0.9262
epoch 20 Accuracy:- 0.9276
epoch 21 Accuracy:- 0.9292
epoch 22 Accuracy:- 0.93
epoch 23 Accuracy:- 0.9319
epoch 24 Accuracy:- 0.933
epoch 25 Accuracy:- 0.934
epoch 26 Accuracy:- 0.9347
epoch 27 Accuracy:- 0.9355
epoch 28 Accuracy:- 0.9367
epoch 29 Accuracy:- 0.9371
epoch 30 Acc

In [15]:
def deepModelWithDropout(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden1, p_keep_hidden2):
    # this network is the same as the previous one except with an extra hidden layer + dropout
    #X->Dropout->h1->Dropout->h2->dropout->output logits
    X = tf.nn.dropout(X, p_keep_input)
    h = tf.nn.relu(tf.matmul(X, w_h))
    h = tf.nn.dropout(h, p_keep_hidden1)
    h2 = tf.nn.relu(tf.matmul(h, w_h2))
    h2 = tf.nn.dropout(h2, p_keep_hidden2)
    return tf.matmul(h2, w_o)

FEATURE_SIZE=784
OUTPUT_SIZE=10
HIDDEN_LAYER_SIZE=625
X = tf.placeholder("float", [None, FEATURE_SIZE])
Y = tf.placeholder("float", [None, OUTPUT_SIZE])

w_h  = init_weights([FEATURE_SIZE, HIDDEN_LAYER_SIZE])
w_h2 = init_weights([HIDDEN_LAYER_SIZE, HIDDEN_LAYER_SIZE])
w_o  = init_weights([HIDDEN_LAYER_SIZE, OUTPUT_SIZE])

p_keep_input   = tf.placeholder("float")
p_keep_hidden1 = tf.placeholder("float")
p_keep_hidden2 = tf.placeholder("float")
pr_y           = deepModelWithDropout(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden1, p_keep_hidden2)

cost       = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pr_y, labels=Y))
train_op   = tf.train.GradientDescentOptimizer(0.05).minimize(cost)
predict_op = tf.argmax(pr_y, 1)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(5):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            sess.run(train_op, feed_dict={X:trX[start:end], Y:trY[start:end], p_keep_input:1.0,
                                         p_keep_hidden1:0.8, p_keep_hidden2:0.5})
        if i%1==0:
            predictions = sess.run(predict_op, feed_dict={X:teX, p_keep_hidden1:1.0, p_keep_input:1.0,
                                                         p_keep_hidden1:1.0, p_keep_hidden2:1.0})
            actuals     = np.argmax(teY, axis=1)
            accuracy    = np.mean(predictions==actuals)
            print("epoch " + str(i) + " Accuracy:- " + str(accuracy))


epoch 0 Accuracy:- 0.785
epoch 1 Accuracy:- 0.8845
epoch 2 Accuracy:- 0.9059
epoch 3 Accuracy:- 0.9198
epoch 4 Accuracy:- 0.9317
